In [70]:
import nltk
import pandas as pd
from nltk.corpus import stopwords
from textblob import Word
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import wordcloud
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [71]:
import pandas as pd
df = pd.read_csv("/content/amazon_alexa.tsv", sep="\t")
df.head()

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1


In [72]:
df['sentiments'] = df['rating'].apply(lambda rating: 1 if rating > 3 else 0)
df

,rating,date,variation,verified_reviews,feedback,sentiments
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1,1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1,1
4,5,31-Jul-18,Charcoal Fabric,Music,1,1
...,...,...,...,...,...,...
3145,5,30-Jul-18,Black Dot,"Perfect for kids, adults and everyone in betwe...",1,1
3146,5,30-Jul-18,Black Dot,"Listening to music, searching locations, check...",1,1
3147,5,30-Jul-18,Black Dot,"I do love these things, i have them running my...",1,1
3148,5,30-Jul-18,White Dot,Only complaint I have is that the sound qualit...,1,1


In [73]:
sentiment = df['sentiments'].values


In [74]:
corpus = df['verified_reviews'].astype(str).values
import nltk
import re
import string

cleaned_corpus = [re.sub(f"[{re.escape(string.punctuation)}]", "", doc) for doc in corpus]


In [75]:
tokenizer = Tokenizer(num_words=500, split=' ')
tokenizer.fit_on_texts(cleaned_corpus)
X = tokenizer.texts_to_sequences(cleaned_corpus)
X = pad_sequences(X)



In [76]:
model = Sequential()
model.add(Embedding(500, 120, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(176, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model.summary())


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_3             │ ?                      │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [91]:

import numpy as np

X_train,X_test,y_train,y_test = train_test_split(X,sentiment, test_size=0.2, random_state=42)
output = y_test
p=[]
for i in y_train:
  if i==0:
    p.append([1,0])
  elif i==1:
    p.append([0,1])
y_train=np.array(p)
q=[]
for i in y_test:
  if i==0:
    q.append([1,0])
  elif i==1:
    q.append([0,1])
y_test=np.array(q)



print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (2520, 390)
y_train shape: (2520, 2)
X_test shape: (630, 390)
y_test shape: (630, 2)


In [86]:
batch_size= 28
model.fit(X_train, y_train, epochs = 10, batch_size=batch_size, verbose = 'auto')

Epoch 1/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 110s 1s/step - accuracy: 0.9684 - loss: 0.0784
Epoch 2/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 104s 1s/step - accuracy: 0.9700 - loss: 0.0833
Epoch 3/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 105s 1s/step - accuracy: 0.9777 - loss: 0.0562
Epoch 4/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 143s 1s/step - accuracy: 0.9760 - loss: 0.0661
Epoch 5/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 105s 1s/step - accuracy: 0.9756 - loss: 0.0543
Epoch 6/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 144s 1s/step - accuracy: 0.9768 - loss: 0.0614
Epoch 7/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 145s 1s/step - accuracy: 0.9836 - loss: 0.0509
Epoch 8/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 139s 1s/step - accuracy: 0.9750 - loss: 0.0525
Epoch 9/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 108s 1s/step - accuracy: 0.9865 - loss: 0.0407
Epoch 10/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 109s 1s/step - accuracy: 0.9786 - loss: 0.0520


In [95]:
model.evaluate(X_test,y_test)

20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 186ms/step - accuracy: 0.9083 - loss: 0.3709


[0.39179497957229614, 0.9095237851142883]

In [96]:
y_predict = model.predict(X_test)

y_approx=[]
for i in y_predict:
  if i[0]>i[1]:
    y_approx.append(0)
  else:
    y_approx.append(1)
result=[]
for i in range(len(y_approx)):
  if y_approx[i]==output[i]:
    result.append("TRUE")
  else:
    result.append("FALSE")
result.count("TRUE")
accuracy = result.count("TRUE")/len(result)
print(accuracy)

20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 194ms/step
0.9095238095238095
